In [24]:
import pandas as pd

In [67]:
df = pd.read_csv('tracking_week_1.csv')
df2 = pd.read_csv("tracking_week_2.csv")
df3 = pd.read_csv("tracking_week_3.csv")
df4 = pd.read_csv("tracking_week_4.csv")
df5 = pd.read_csv("tracking_week_5.csv")
df6 = pd.read_csv("tracking_week_6.csv")
df7 = pd.read_csv("tracking_week_7.csv")
df8 = pd.read_csv("tracking_week_8.csv")
df9 = pd.read_csv("tracking_week_9.csv")
players = pd.read_csv('players.csv')

In [68]:
dfs = [df, df2, df3, df4, df5, df6, df7, df8, df9]

df = pd.concat(dfs, axis=0, ignore_index=True)

In [61]:
positions = pd.merge(
    df[['gameId','playId','nflId','y']][df['frameType']=='SNAP'],
    players[['nflId','position']],
    how = "left",
    on = 'nflId'
)


In [62]:
lineman = ['T','C','G','FB']

In [63]:
oline = positions[positions['position'].isin(lineman)]

In [64]:
linecoords = pd.DataFrame(columns=['gameId','playId','topOfLine','bottomOfLine'])
for _, i in oline.iterrows():
    inplay = ((linecoords['gameId'] == i['gameId']) & (linecoords['playId'] == i['playId'])).any()
    
    if not inplay:
        play = oline[(oline['gameId'] == i['gameId']) & 
                           (oline['playId'] == i['playId'])]
        
        thisplay = []
        thisplay.extend([
            i['gameId'],
            i['playId'],
        ])


        max = 0
        min = float("inf")
        for _, j in play.iterrows():
            if j['y'] > max:
                max = j['y']
            elif j['y'] < min:
                min = j['y']

        thisplay.extend([
            max,
            min
        ])

        new_row = pd.DataFrame([thisplay], columns=linecoords.columns)
        linecoords = pd.concat([linecoords, new_row], ignore_index=True)

C:\Users\bhaga\AppData\Local\Temp\ipykernel_33440\1092303416.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  linecoords = pd.concat([linecoords, new_row], ignore_index=True)


In [ ]:
linecoords.to_csv("linecoods.csv")

In [79]:
playerCoords = df[['gameId','playId','nflId','x','y']][df['frameType']=="SNAP"]

In [80]:
playerCoords = pd.merge(
    playerCoords,
    players[['nflId','position']],
    how="left",
    on=['nflId']
)

In [84]:
playerCoords = playerCoords[(playerCoords['position']=="FS")|(playerCoords['position']=="SS")]

In [83]:
plays = pd.read_csv("plays.csv")

In [88]:
playerCoords = pd.merge(
    playerCoords,
    plays[['gameId','playId','absoluteYardlineNumber']],
    how = "left",
    on=['gameId','playId']
)

In [91]:
safetyPos = pd.DataFrame(columns=['gameId','playId','safetyTop','safetyBottom'])
for _, i in playerCoords.iterrows():
    inplay = ((safetyPos['gameId'] == i['gameId']) & (safetyPos['playId'] == i['playId'])).any()
    
    if not inplay:
        play = playerCoords[(playerCoords['gameId'] == i['gameId']) & 
                           (playerCoords['playId'] == i['playId'])]
        
        thisplay = []
        thisplay.extend([
            i['gameId'],
            i['playId'],
        ])


        top = 0
        bottom = 0
        for _, j in play.iterrows():
            if j['y'] > 26.6:
                top += 1
            elif j['y'] <= 26.6:
                bottom += 1

        thisplay.extend([
            top,
            bottom
        ])

        new_row = pd.DataFrame([thisplay], columns=safetyPos.columns)
        safetyPos = pd.concat([safetyPos, new_row], ignore_index=True)

In [93]:
safetyPos.to_csv("safetyPositions")